In [82]:
import os
import numpy as np

if not os.path.exists("data/"):
  !mkdir data

if not os.path.exists("data/chip1.bin"):
  if not os.path.exists("data/SRAM_data.zip"):
    !wget https://users.fit.cvut.cz/bucekj/SRAM_data.zip -O SRAM_data.zip
  !unzip SRAM_data.zip -d data
  !rm SRAM_data.zip

In [81]:
# Load Datafiles

MEASUREMENTS = 1000
MEM_SIZE = 512*8
CHIPS = 10

rams = np.zeros([CHIPS,MEASUREMENTS,MEM_SIZE], dtype=np.uint8)
for i in range(CHIPS):
    rams[i] = np.reshape(np.unpackbits(np.fromfile(f'data/chip{i+1}.bin', dtype='uint8')), ([MEASUREMENTS, MEM_SIZE]))

In [ ]:
print(rams, rams.shape, rams.dtype)

In [61]:
# means -> "stability of bits"
means = rams.mean(axis=1, keepdims=True)

# Reference responses
refs = means.round().astype(int)

In [ ]:
print(means)
print(means.shape)

In [62]:
# Intra-Hamming distance
# XOR reference with the acutal measurement and create an average from it
hd_intra = np.mean(rams ^ refs)
print('HD_intra =', hd_intra ,'~',(hd_intra*100).astype(int),'%')

HD_intra = 0.030485107421875 ~ 3 %


In [63]:
# Inter-Hamming distance
# XOR reference of a chip with references of the other chips
xor_ref = refs ^ refs.transpose([1,0,2])
hd_inter = np.sum(xor_ref)/(CHIPS*(CHIPS-1)*MEM_SIZE)
print('HD_inter =', hd_inter ,'~',(hd_inter*100).astype(int),'%')

HD_inter = 0.43573133680555554 ~ 43 %


In [64]:
#  Mask of stable bits - Black magic
stable = ((means < 0.05) | (means > 0.95))
masked_refs = np.zeros([10,1,1024],dtype='uint8')
masked_rams = np.zeros([10,1000,1024],dtype='uint8')
for i in range (10):
    index = np.nonzero(stable[i])
    masked_refs[i] = refs[i][index][:1024]
    masked_rams[i] = rams[i][:,index[1][:1024]]

In [65]:
# Intra-Hamming distance for the masked PUF
# XOR reference with the acutal measurement and create an average from it
hd_intra_masked = np.mean(masked_rams ^ masked_refs)
print('HD_intra_masked =', hd_intra_masked ,'~',(hd_intra_masked*100).astype(int),'%')

HD_intra_masked = 0.0017783203125 ~ 0 %


In [66]:
# Inter-Hamming distance for the masked PUF
# XOR reference of a chip with references of the other chips
xor_masked_ref = masked_refs ^ masked_refs.transpose([1,0,2])
hd_inter_masked = np.sum(xor_masked_ref)/(CHIPS*(CHIPS-1)*1024)
print('HD_inter_masked =', hd_inter_masked ,'~',(hd_inter_masked*100).astype(int),'%')

HD_inter_masked = 0.4957682291666667 ~ 49 %


In [67]:
# Mask stable bits - best selection
PUF_bits = 1024

masked_refs_best = np.zeros([CHIPS,1,PUF_bits],dtype='uint8')
masked_rams_best = np.zeros([CHIPS,MEASUREMENTS,PUF_bits],dtype='uint8')
masks = np.zeros([CHIPS,1,MEM_SIZE],dtype='uint8')

# Iterate through each row
for i in range(CHIPS):
    row = means[i, 0]  # Select the current row

    # Find indices of values closest to either 0 or 1
    indices_0 = np.argsort(np.minimum(row, 1 - row))[:PUF_bits]

    # Set the selected indices to 1
    masks[i, 0, indices_0] = 1

    # Chose masked bits for reference
    selected_values = refs[i, 0, indices_0]
    masked_refs_best[i, 0, :] = selected_values

    # Chose masked bits from rams for PUF
    selected_values2 = rams[i, : , indices_0]
    masked_rams_best[i, :, :] = selected_values2.transpose()

In [68]:
# Save masks into file
masks =  np.squeeze(masks, axis=1)
np.savetxt('masks.txt', masks.astype(int), fmt='%d', delimiter='')

In [ ]:
print(masked_refs_best.shape)
print(masked_refs_best)

In [ ]:
print(masked_rams_best.shape)
print(masked_rams_best)

In [ ]:
print(masks.shape)
print(masks)

In [69]:
# Intra-Hamming distance for the masked PUF - best selection
# XOR reference with the acutal measurement and create an average from it
hd_intra_masked_best = np.mean(masked_rams_best ^ masked_refs_best)
print('HD_intra_masked_best =', hd_intra_masked_best ,'~',(hd_intra_masked_best*100).astype(int),'%')

HD_intra_masked_best = 0.0 ~ 0 %


In [70]:
# Inter-Hamming distance for the masked PUF - best selection
# XOR reference of a chip with references of the other chips
xor_masked_ref_best = masked_refs_best ^ masked_refs_best.transpose([1,0,2])
hd_inter_masked_best = np.sum(xor_masked_ref_best)/(CHIPS*(CHIPS-1)*1024)
print('HD_inter_masked_best =', hd_inter_masked_best ,'~',(hd_inter_masked_best*100).astype(int),'%')

HD_inter_masked_best = 0.5017795138888889 ~ 50 %


In [71]:
# Print RESULTS
print('HD_intra =', hd_intra ,'~',(hd_intra*100).astype(int),'%')
print('HD_inter =', hd_inter ,'~',(hd_inter*100).astype(int),'%')
print('HD_intra_masked =', hd_intra_masked ,'~',(hd_intra_masked*100).astype(int),'%')
print('HD_inter_masked =', hd_inter_masked ,'~',(hd_inter_masked*100).astype(int),'%')
print('HD_intra_masked_best =', hd_intra_masked_best ,'~',(hd_intra_masked_best*100).astype(int),'%')
print('HD_inter_masked_best =', hd_inter_masked_best ,'~',(hd_inter_masked_best*100).astype(int),'%')

HD_intra = 0.030485107421875 ~ 3 %
HD_inter = 0.43573133680555554 ~ 43 %
HD_intra_masked = 0.0017783203125 ~ 0 %
HD_inter_masked = 0.4957682291666667 ~ 49 %
HD_intra_masked_best = 0.0 ~ 0 %
HD_inter_masked_best = 0.5017795138888889 ~ 50 %
